# ДЗ 4

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
src_path = os.path.join(module_path, "src")
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(src_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [2]:
from src.recommenders import MainRecommender

In [3]:
data = pd.read_csv('./sem_2/retail_train.csv')   # '../data/transaction_data.csv'

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                     'product_id': 'item_id'},
            inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] <  data['week_no'].max() - test_size_weeks]
data_test  = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
recommender = MainRecommender(data_train)

---

### get_similar_items_recommendation

In [6]:
popularity = recommender.prep_data.loc[recommender.prep_data["user_id"]==2222, ['item_id', 'quantity']].\
                groupby(['item_id'])['quantity'].count().reset_index()
popularity.sort_values('quantity', ascending=False, inplace=True)
top_item_list = popularity.loc[popularity["item_id"]!=999999, "item_id"][:5].tolist()
top_item_list

[5569230, 995242, 870780, 1101010, 951590]

In [7]:
for item in top_item_list:
    print(recommender.id_to_itemid[recommender.model.similar_items(recommender.itemid_to_id[item], N=1)[0][0]])

5569230
995242
870780
1101010
951590


In [8]:
rec = set()

In [9]:
for item in top_item_list:
    rec.add(recommender.id_to_itemid[recommender.model.similar_items(recommender.itemid_to_id[item], N=1)[0][0]])

In [10]:
rec

{870780, 951590, 995242, 1101010, 5569230}

In [11]:
recommender.get_similar_items_recommendation(user=2222)

{870780, 951590, 995242, 1101010, 5569230}

---

### get_similar_users_recommendation

In [23]:
user_id = 2425

In [24]:
sim_users_list = [recommender.id_to_userid[usr] for usr, p in recommender.model.similar_users(recommender.userid_to_id[user_id], 
                                                                                              N=5)]
sim_users_list

[2425, 817, 1484, 1676, 409]

In [25]:
sim_users_list = [recommender.id_to_userid[usr] for usr, p in recommender.model.similar_users(recommender.userid_to_id[user_id], 
                                                                                              N=5+1)[1:5+1]]
sim_users_list

[817, 1484, 1676, 409, 1078]

In [14]:
rec = set()

In [15]:
for user in sim_users_list:
    rec.add(list(recommender.get_recommendations(user, rec_num=1))[0])

In [16]:
rec

{1041259, 1058997, 1082185, 1126899}

In [17]:
recommender.get_similar_users_recommendation(user_id)

{1041259, 1058997, 1082185, 1126899, 1127831}